## TAREA 3

In [3]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=30b27e6dda37d48b568c66ef848fc82fb167e1971c8a80230d83bd785f41bca5
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [4]:
import pyspark
print(pyspark.__version__)

3.5.3


In [5]:
pip install findspark

In [44]:
import os
import sys
import findspark
import pandas as pd
findspark.init()
findspark.find()
from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F
import time

In [72]:
from pyspark import SparkContext
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType
from pyspark.sql.functions import mean, max
from pyspark.sql import functions as F
from pyspark.sql.functions import col
from pyspark.sql.functions import lit

In [9]:
import findspark
findspark.init()

In [10]:
import requests

api_key = "73e3e0580fb16a75fdf023438b452211"
city = "Guerrero"
url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"

response = requests.get(url)
data = response.json()

print(data)


{'coord': {'lon': -100, 'lat': 17.6667}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'base': 'stations', 'main': {'temp': 17.72, 'feels_like': 17.95, 'temp_min': 17.72, 'temp_max': 17.72, 'pressure': 1020, 'humidity': 92, 'sea_level': 1020, 'grnd_level': 827}, 'visibility': 10000, 'wind': {'speed': 1.9, 'deg': 181, 'gust': 1.77}, 'rain': {'1h': 0.19}, 'clouds': {'all': 46}, 'dt': 1728873935, 'sys': {'country': 'MX', 'sunrise': 1728822763, 'sunset': 1728865149}, 'timezone': -21600, 'id': 3527213, 'name': 'Guerrero', 'cod': 200}


In [11]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("OpenWeatherData") \
    .getOrCreate()

In [12]:
weather_data = {
    'city': data['name'],
    'temperature': data['main']['temp'],
    'description': data['weather'][0]['description'],
    'humidity': data['main']['humidity']
}

data_list = [weather_data]

rdd = spark.sparkContext.parallelize(data_list)

print(rdd.collect())


[{'city': 'Guerrero', 'temperature': 17.72, 'description': 'light rain', 'humidity': 92}]


Para considerar varias ciudades:

In [21]:
cities = [
    "Aguascalientes, Mexico",
    "Mexicali, Mexico",
    "La Paz, Mexico",
    "Campeche, Mexico",
    "Tuxtla Gutiérrez, Mexico",
    "Chihuahua, Mexico",
    "Saltillo, Mexico",
    "Colima, Mexico",
    "Durango, Mexico",
    "Guanajuato, Mexico",
    "Chilpancingo, Mexico",
    "Pachuca, Mexico",
    "Guadalajara, Mexico",
    "Toluca, Mexico",
    "Morelia, Mexico",
    "Cuernavaca, Mexico",
    "Tepic, Mexico",
    "Monterrey, Mexico",
    "Oaxaca de Juárez, Mexico",
    "Puebla, Mexico",
    "Querétaro, Mexico",
    "Chetumal, Mexico",
    "San Luis Potosí, Mexico",
    "Culiacán, Mexico",
    "Hermosillo, Mexico",
    "Villahermosa, Mexico",
    "Ciudad Victoria, Mexico",
    "Tlaxcala, Mexico",
    "Xalapa, Mexico",
    "Mérida, Mexico",
    "Zacatecas, Mexico"
]

In [22]:
sc = SparkContext.getOrCreate()
weather_data_list = []

for city in cities:
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        weather_data = {
            'city': data['name'],
            'temperature': data['main']['temp'],
            'description': data['weather'][0]['description'],
            'humidity': data['main']['humidity']
        }
        weather_data_list.append(weather_data)
    else:
        print(f"Error fetching weather for {city}: {response.status_code}")

rdd = sc.parallelize(weather_data_list)
print(rdd.collect())

[{'city': 'Aguascalientes', 'temperature': 14.97, 'description': 'clear sky', 'humidity': 58}, {'city': 'Mexicali', 'temperature': 28.67, 'description': 'clear sky', 'humidity': 10}, {'city': 'La Paz', 'temperature': 10.32, 'description': 'few clouds', 'humidity': 56}, {'city': 'Campeche', 'temperature': 23.88, 'description': 'overcast clouds', 'humidity': 82}, {'city': 'Tuxtla Gutiérrez', 'temperature': 22.45, 'description': 'overcast clouds', 'humidity': 95}, {'city': 'Chihuahua', 'temperature': 19.46, 'description': 'clear sky', 'humidity': 21}, {'city': 'Saltillo', 'temperature': 15.72, 'description': 'clear sky', 'humidity': 59}, {'city': 'Colima', 'temperature': 20.54, 'description': 'few clouds', 'humidity': 94}, {'city': 'Durango', 'temperature': 13.42, 'description': 'clear sky', 'humidity': 42}, {'city': 'Guanajuato City', 'temperature': 18.44, 'description': 'broken clouds', 'humidity': 67}, {'city': 'Chilpancingo', 'temperature': 17.86, 'description': 'moderate rain', 'humi

In [39]:
inicio = time.time()
city_descr_rdd = rdd.map(lambda x: (x['description'], x['city']))
print(city_descr_rdd.collect())
fin = time.time() # Fin de ejecución

print("Tiempo sin cache:", fin - inicio)


[('clear sky', 'Aguascalientes'), ('clear sky', 'Mexicali'), ('few clouds', 'La Paz'), ('overcast clouds', 'Campeche'), ('overcast clouds', 'Tuxtla Gutiérrez'), ('clear sky', 'Chihuahua'), ('clear sky', 'Saltillo'), ('few clouds', 'Colima'), ('clear sky', 'Durango'), ('broken clouds', 'Guanajuato City'), ('moderate rain', 'Chilpancingo'), ('broken clouds', 'Pachuca'), ('clear sky', 'Guadalajara'), ('broken clouds', 'Toluca'), ('broken clouds', 'Morelia'), ('broken clouds', 'Cuernavaca'), ('few clouds', 'Tepic'), ('clear sky', 'Monterrey'), ('broken clouds', 'Oaxaca City'), ('overcast clouds', 'Puebla'), ('scattered clouds', 'Querétaro'), ('light rain', 'Chetumal'), ('clear sky', 'San Luis Potosí'), ('few clouds', 'Culiacán'), ('clear sky', 'Hermosillo'), ('broken clouds', 'Villahermosa'), ('clear sky', 'Ciudad Victoria'), ('overcast clouds', 'Tlaxcala'), ('broken clouds', 'Xalapa'), ('clear sky', 'Mérida'), ('clear sky', 'Zacatecas')]
Tiempo sin cache: 0.47708892822265625


In [33]:
grouped_by_description = city_descr_rdd.groupByKey()
result = grouped_by_description.collect()
for description, cities in result:
    print(f"Description: {description}")
    print(f"Cities: {', '.join(cities)}")

Description: clear sky
Cities: Aguascalientes, Mexicali, Chihuahua, Saltillo, Durango, Guadalajara, Monterrey, San Luis Potosí, Hermosillo, Ciudad Victoria, Mérida, Zacatecas
Description: broken clouds
Cities: Guanajuato City, Pachuca, Toluca, Morelia, Cuernavaca, Oaxaca City, Villahermosa, Xalapa
Description: moderate rain
Cities: Chilpancingo
Description: scattered clouds
Cities: Querétaro
Description: few clouds
Cities: La Paz, Colima, Tepic, Culiacán
Description: overcast clouds
Cities: Campeche, Tuxtla Gutiérrez, Puebla, Tlaxcala
Description: light rain
Cities: Chetumal


In [45]:
df_spark = spark.createDataFrame(rdd)
df_pandas = df_spark.toPandas()

In [46]:
df_pandas.head(3)

,city,description,humidity,temperature
0,Aguascalientes,clear sky,58,14.97
1,Mexicali,clear sky,10,28.67
2,La Paz,few clouds,56,10.32


In [47]:
df_pandas.tail(3)

,city,description,humidity,temperature
28,Xalapa,broken clouds,99,14.68
29,Mérida,clear sky,83,26.01
30,Zacatecas,clear sky,48,11.81


In [48]:
df_pandas.describe()

,humidity,temperature
count,31.000000,31.000000
mean,68.032258,19.093871
std,21.592412,6.062941
min,10.000000,10.320000
25%,58.500000,14.100000
50%,70.000000,17.860000
75%,83.000000,23.300000
max,99.000000,33.440000


In [51]:
print(f"Mínimo: {df_pandas['temperature'].min()}")
print(f"Máximo: {df_pandas['temperature'].max()}")
print(f"Media: {df_pandas['temperature'].mean()}")
print(f"Mediana: {df_pandas['temperature'].median()}")
print(f"Desviación estándar: {df_pandas['temperature'].std()}")

print('Correlación')
df_pandas[['temperature', 'humidity']].corr()

Mínimo: 10.32
Máximo: 33.44
Media: 19.093870967741932
Mediana: 17.86
Desviación estándar: 6.0629413529624685
Correlación


,temperature,humidity
temperature,1.000000,-0.181286
humidity,-0.181286,1.000000


In [54]:
schema = StructType([
    StructField("city", StringType(), True),
    StructField("temperature", FloatType(), True),
    StructField("description", StringType(), True),
    StructField("humidity", IntegerType(), True)
])

pysparkDF = spark.createDataFrame(data=weather_data_list, schema=schema)

pysparkDF.printSchema()

root
 |-- city: string (nullable = true)
 |-- temperature: float (nullable = true)
 |-- description: string (nullable = true)
 |-- humidity: integer (nullable = true)



In [55]:
pysparkDF.show(truncate = False)

+----------------+-----------+---------------+--------+
|city            |temperature|description    |humidity|
+----------------+-----------+---------------+--------+
|Aguascalientes  |14.97      |clear sky      |58      |
|Mexicali        |28.67      |clear sky      |10      |
|La Paz          |10.32      |few clouds     |56      |
|Campeche        |23.88      |overcast clouds|82      |
|Tuxtla Gutiérrez|22.45      |overcast clouds|95      |
|Chihuahua       |19.46      |clear sky      |21      |
|Saltillo        |15.72      |clear sky      |59      |
|Colima          |20.54      |few clouds     |94      |
|Durango         |13.42      |clear sky      |42      |
|Guanajuato City |18.44      |broken clouds  |67      |
|Chilpancingo    |17.86      |moderate rain  |94      |
|Pachuca         |12.44      |broken clouds  |88      |
|Guadalajara     |17.87      |clear sky      |63      |
|Toluca          |13.39      |broken clouds  |62      |
|Morelia         |13.52      |broken clouds  |76

In [69]:
df_av = pysparkDF.select(F.mean("temperature"), F.mean("humidity")).show()

+------------------+-----------------+
|  avg(temperature)|    avg(humidity)|
+------------------+-----------------+
|19.093870962819746|68.03225806451613|
+------------------+-----------------+



In [71]:
df_fahr = pysparkDF.withColumn("temperature", col("temperature") * 9/5+32)
df_fahr.show()

+----------------+------------------+---------------+--------+
|            city|       temperature|    description|humidity|
+----------------+------------------+---------------+--------+
|  Aguascalientes| 58.94599914550781|      clear sky|      58|
|        Mexicali| 83.60599975585937|      clear sky|      10|
|          La Paz|  50.5759994506836|     few clouds|      56|
|        Campeche| 74.98399963378907|overcast clouds|      82|
|Tuxtla Gutiérrez| 72.41000061035156|overcast clouds|      95|
|       Chihuahua| 67.02799682617187|      clear sky|      21|
|        Saltillo|60.295999145507814|      clear sky|      59|
|          Colima| 68.97200317382813|     few clouds|      94|
|         Durango|56.155999755859376|      clear sky|      42|
| Guanajuato City| 65.19200134277344|  broken clouds|      67|
|    Chilpancingo| 64.14800109863282|  moderate rain|      94|
|         Pachuca| 54.39199981689453|  broken clouds|      88|
|     Guadalajara| 64.16600036621094|      clear sky|  

In [74]:
pysparkDF.withColumn(
    "country",
    lit("MEX")
).show()

+----------------+-----------+---------------+--------+-------+
|            city|temperature|    description|humidity|country|
+----------------+-----------+---------------+--------+-------+
|  Aguascalientes|      14.97|      clear sky|      58|    MEX|
|        Mexicali|      28.67|      clear sky|      10|    MEX|
|          La Paz|      10.32|     few clouds|      56|    MEX|
|        Campeche|      23.88|overcast clouds|      82|    MEX|
|Tuxtla Gutiérrez|      22.45|overcast clouds|      95|    MEX|
|       Chihuahua|      19.46|      clear sky|      21|    MEX|
|        Saltillo|      15.72|      clear sky|      59|    MEX|
|          Colima|      20.54|     few clouds|      94|    MEX|
|         Durango|      13.42|      clear sky|      42|    MEX|
| Guanajuato City|      18.44|  broken clouds|      67|    MEX|
|    Chilpancingo|      17.86|  moderate rain|      94|    MEX|
|         Pachuca|      12.44|  broken clouds|      88|    MEX|
|     Guadalajara|      17.87|      clea

In [76]:
pysparkDF.filter(pysparkDF.temperature > 20).show(truncate=False)

+----------------+-----------+---------------+--------+
|city            |temperature|description    |humidity|
+----------------+-----------+---------------+--------+
|Mexicali        |28.67      |clear sky      |10      |
|Campeche        |23.88      |overcast clouds|82      |
|Tuxtla Gutiérrez|22.45      |overcast clouds|95      |
|Colima          |20.54      |few clouds     |94      |
|Tepic           |22.72      |few clouds     |83      |
|Monterrey       |26.43      |clear sky      |49      |
|Chetumal        |27.4       |light rain     |77      |
|Culiacán        |29.82      |few clouds     |70      |
|Hermosillo      |33.44      |clear sky      |32      |
|Villahermosa    |24.45      |broken clouds  |87      |
|Ciudad Victoria |21.47      |clear sky      |60      |
|Mérida          |26.01      |clear sky      |83      |
+----------------+-----------+---------------+--------+



In [77]:
pysparkDF.filter( ((pysparkDF.temperature > 20) & (pysparkDF.humidity  >50)) ).show(truncate=False)

+----------------+-----------+---------------+--------+
|city            |temperature|description    |humidity|
+----------------+-----------+---------------+--------+
|Campeche        |23.88      |overcast clouds|82      |
|Tuxtla Gutiérrez|22.45      |overcast clouds|95      |
|Colima          |20.54      |few clouds     |94      |
|Tepic           |22.72      |few clouds     |83      |
|Chetumal        |27.4       |light rain     |77      |
|Culiacán        |29.82      |few clouds     |70      |
|Villahermosa    |24.45      |broken clouds  |87      |
|Ciudad Victoria |21.47      |clear sky      |60      |
|Mérida          |26.01      |clear sky      |83      |
+----------------+-----------+---------------+--------+



In [78]:
pysparkDF.filter(pysparkDF.description.contains("clouds")).show()

+----------------+-----------+----------------+--------+
|            city|temperature|     description|humidity|
+----------------+-----------+----------------+--------+
|          La Paz|      10.32|      few clouds|      56|
|        Campeche|      23.88| overcast clouds|      82|
|Tuxtla Gutiérrez|      22.45| overcast clouds|      95|
|          Colima|      20.54|      few clouds|      94|
| Guanajuato City|      18.44|   broken clouds|      67|
|         Pachuca|      12.44|   broken clouds|      88|
|          Toluca|      13.39|   broken clouds|      62|
|         Morelia|      13.52|   broken clouds|      76|
|      Cuernavaca|      16.43|   broken clouds|      81|
|           Tepic|      22.72|      few clouds|      83|
|     Oaxaca City|       16.3|   broken clouds|      85|
|          Puebla|      17.47| overcast clouds|      68|
|       Querétaro|      14.96|scattered clouds|      70|
|        Culiacán|      29.82|      few clouds|      70|
|    Villahermosa|      24.45| 

In [80]:
pysparkDF.sort("city", ascending = [True]).show()

+---------------+-----------+---------------+--------+
|           city|temperature|    description|humidity|
+---------------+-----------+---------------+--------+
| Aguascalientes|      14.97|      clear sky|      58|
|       Campeche|      23.88|overcast clouds|      82|
|       Chetumal|       27.4|     light rain|      77|
|      Chihuahua|      19.46|      clear sky|      21|
|   Chilpancingo|      17.86|  moderate rain|      94|
|Ciudad Victoria|      21.47|      clear sky|      60|
|         Colima|      20.54|     few clouds|      94|
|     Cuernavaca|      16.43|  broken clouds|      81|
|       Culiacán|      29.82|     few clouds|      70|
|        Durango|      13.42|      clear sky|      42|
|    Guadalajara|      17.87|      clear sky|      63|
|Guanajuato City|      18.44|  broken clouds|      67|
|     Hermosillo|      33.44|      clear sky|      32|
|         La Paz|      10.32|     few clouds|      56|
|       Mexicali|      28.67|      clear sky|      10|
|      Mon